In [ ]:
import os
import re
from textblob import TextBlob
from nltk.corpus import wordnet as wn

def train_at():
	term_sentence=dict()
	at=[]
	lines = [line.split(',') for line in open('Train_restaurant.csv')]	#read train file
	for i in lines:
		if i[0]=="":	#when no aspect term, read next line
			continue
		else:
			str1=i[2:]		#cause my computer seperate each sentence by ",", so from the third line to the end is the sentence
			str2=""
			for k in str1:	
				str2+=k
				str2+=","
			str2=str2[0:-2]	#ignore the enter(\n)
			
			list=re.findall("\((\w+)\#",i[0])	#extract the aspect term from the form of a#b#c...
			for j in list:
				if j not in term_sentence:
					term_sentence[j]=str2
			list=re.findall("\((\w+\s\w+)\#",i[0])
			for j in list:
				if j not in term_sentence:
					term_sentence[j]=str2
			list=re.findall("\((\w+\s\w+\s\w+)\#",i[0])
			for j in list:
				if j not in term_sentence:
					term_sentence[j]=str2
			list=re.findall("\((\w+\s\w+\s\w+\s\w+)\#",i[0])
			for j in list:
				if j not in term_sentence:
					term_sentence[j]=str2
			list=re.findall("\((\w+\s\w+\s\w+\s\w+\s\w+)\#",i[0])
			for j in list:
				if j not in term_sentence:
					term_sentence[j]=str2
	f = open('mark.csv', 'w')	#store the aspect term(first column is the AT, second column is empty for AC, from the third column to the end is the sentence)
	for i in term_sentence:
		f.write(i)
		f.write(",")
		f.write("")
		f.write(",")
		f.write(term_sentence[i])
		f.write("\n")
	f.close()	
	for i in term_sentence:	#return AT
		at.append(i)
	return at
	
def test_at(at_list1):
	at=dict()
	lines = [line.split(',') for line in open('Test_restaurant_PhaseA.csv')]	#read test file
	for i in lines:
		str1=i[0:]
		str2=""
		for k in str1:
			str2+=k
			str2+=","
		str2=str2[0:-2]
		
		str3=TextBlob(str2)				#use TextBlob to find each sentence's noun and noun phrases
		noun_phrases=str3.noun_phrases
		for j in noun_phrases:	#count the frequency for each noun and noun phrases
			j=str(j)
			if j not in at:
				at[j]=1
			else:
				at[j]+=1
	fre_at=[]
	for i in at:
		if at[i]>=2:	#keep the term whose frequency >=2
			fre_at.append(i)
		else:
			terms=i.split(" ")
			for k in terms:	#check each phrases' noun 
				if k in at_list1:	#if part of the phrases in the AT from train_restaurant, we will keep it
					fre_at.append(i)
					break
	return fre_at
		
def adj_ac():
	ac_food=[]
	ac_service=[]
	ac_price=[]
	ac_ambience=[]
	ac_others=[]
	lines = [line.split(',') for line in open('Train_restaurant.csv')]	
	for i in lines:
		str1=i[2:]
		str2=""
		for k in str1:
			str2+=k
			str2+=","
		str2=str2[0:-2]
		str3=TextBlob(str2)
		tags=str3.tags		#use TextBlob to tag each sentence's words
		for j in tags:
			if j[1] in ("JJ" , "JJR" , "JJS"):	#if one word is adj
				list=re.findall("\((\w+)\#",i[1])	#use regular expression to find the AC of each sentence
				for m in list:	#put each sentence's adjs in all the category. eg:two adjs, three AC, we will put 2*3=6 times
					if m=="food" and j[0]not in ac_food:
						ac_food.append(j[0])
					elif m=="service" and j[0]not in ac_service:
						ac_service.append(j[0])
					elif m=="price"and j[0]not in ac_price:
						ac_price.append(j[0])
					elif m=="ambience" and j[0]not in ac_ambience:
						ac_ambience.append(j[0])
					elif j[0]not in ac_others:
						ac_others.append(j[0])
	f = open('ac_food.csv', 'w')	#store the adjs. And then, we will delete each .csv file's adj handful.
	for i in ac_food:
		f.write(i)
		f.write("\n")
	f.close()
	f = open('ac_service.csv', 'w')
	for i in ac_service:
		f.write(i)
		f.write("\n")
	f.close()
	f = open('ac_price.csv', 'w')
	for i in ac_price:
		f.write(i)
		f.write("\n")
	f.close()
	f = open('ac_ambience.csv', 'w')
	for i in ac_ambience:
		f.write(i)
		f.write("\n")
	f.close()
	f = open('ac_others.csv', 'w')
	for i in ac_others:
		f.write(i)
		f.write("\n")
	f.close()
	
def adj_wordNet():
	ac_food=[]
	ac_service=[]
	ac_price=[]
	ac_ambience=[]
	ac_others=[]
	lines = [line.split(',') for line in open('ac_food_marked.csv')]	
	for i in lines:	#for each adj
		adj=i[0][:-1]
		adj_synsets=wn.synsets(adj,pos=wn.ADJ)	#find each adj's synsets
		for j in adj_synsets:
			j=str(j)	#turn the result to string 
			list=re.findall("\'(\w+)\.",j)	#and then use regular regression to extract useful adjs
			if len(list)!=0:
				if list[0] not in ac_food:
					ac_food.append(list[0])
	f = open('ac_food_synsets.csv', 'w')	#store
	for i in ac_food:
		f.write(i)
		f.write("\n")
	f.close()

	lines = [line.split(',') for line in open('ac_service_marked.csv')]	
	for i in lines:
		adj=i[0][:-1]
		adj_synsets=wn.synsets(adj,pos=wn.ADJ)
		for j in adj_synsets:
			j=str(j)
			list=re.findall("\'(\w+)\.",j)
			if len(list)!=0:
				if list[0] not in ac_food:
					ac_service.append(list[0])
	f = open('ac_service_synsets.csv', 'w')
	for i in ac_service:
		f.write(i)
		f.write("\n")
	f.close()

	lines = [line.split(',') for line in open('ac_price_marked.csv')]	
	for i in lines:
		adj=i[0][:-1]
		adj_synsets=wn.synsets(adj,pos=wn.ADJ)
		for j in adj_synsets:
			j=str(j)
			list=re.findall("\'(\w+)\.",j)
			if len(list)!=0:
				if list[0] not in ac_price:
					ac_price.append(list[0])
	f = open('ac_price_synsets.csv', 'w')
	for i in ac_price:
		f.write(i)
		f.write("\n")
	f.close()

	lines = [line.split(',') for line in open('ac_ambience_marked.csv')]	
	for i in lines:
		adj=i[0][:-1]
		adj_synsets=wn.synsets(adj,pos=wn.ADJ)
		for j in adj_synsets:
			j=str(j)
			list=re.findall("\'(\w+)\.",j)
			if len(list)!=0:
				if list[0] not in ac_ambience:
					ac_ambience.append(list[0])
	f = open('ac_ambience_synsets.csv', 'w')
	for i in ac_ambience:
		f.write(i)
		f.write("\n")
	f.close()

	lines = [line.split(',') for line in open('ac_others_marked.csv')]	
	for i in lines:
		adj=i[0][:-1]
		adj_synsets=wn.synsets(adj,pos=wn.ADJ)
		for j in adj_synsets:
			j=str(j)
			list=re.findall("\'(\w+)\.",j)
			if len(list)!=0:
				if list[0] not in ac_others:
					ac_others.append(list[0])
	f = open('ac_others_synsets.csv', 'w')
	for i in ac_others:
		f.write(i)
		f.write("\n")
	f.close()	

def check_at(at_list):
	ac_food=[]
	ac_service=[]
	ac_price=[]
	ac_ambience=[]
	ac_others=[]
	lines = [line.split(',') for line in open('ac_food_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_food.append(adj)
	lines = [line.split(',') for line in open('ac_service_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_service.append(adj)
	lines = [line.split(',') for line in open('ac_price_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_price.append(adj)
	lines = [line.split(',') for line in open('ac_ambience_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_ambience.append(adj)
	lines = [line.split(',') for line in open('ac_others_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_others.append(adj)
		
	ac_all=ac_food+ac_service+ac_price+ac_ambience+ac_others	
	at=dict()
	lines = [line.split(',') for line in open('Test_restaurant_PhaseA.csv')]	
	for i in lines:
		str1=i[0:]
		str2=""
		for k in str1:
			str2+=k
			str2+=","
		str2=str2[0:-2]
		at[str2]=[]
		str3=TextBlob(str2)
		noun_phrases=str3.noun_phrases
		for j in noun_phrases:
			j=str(j)
			if j in at_list:	#if one sentence have some nouns which have been marked as AT, then store them.
				at[str2].append(j)
	
	for i in at:
		if len(at[i])==0:	#if one sentence have no AT
			at[i]=[]
			sent=TextBlob(i)	#tag this sentence
			tags=sent.tags
			for j in range(0,len(tags)):
				if tags[j][1] in ("JJ" , "JJR" , "JJS")and tags[j][0] in ac_all:	#if one word is adj and it belongs to one of the ac_synsets
					noun_phrases=sent.noun_phrases	#then find this sentence's nouns
					for k in noun_phrases:
						k=str(k)
						m=k.split(" ")
						for n in range(0,len(tags)):	#if this adj is three words ahead of the noun or behind of the noun, or in the noun phrases, store the noun
							if tags[n][0]==m[0] and n-j<3 and n-j>0:
								for p in m:
									at[i].append(p)
							elif tags[n][0]==m[-1] and j-n<3 and j-n>0:
								for p in m:
									at[i].append(p)
							elif tags[n][0]==m[0] and j-n<=len(m):
								for p in m:
									at[i].append(p)
	for i in at:
		at[i]=set(at[i])
		at[i]=list(at[i])
	at_str=dict()	#adjust the form for storing
	for i in at:
		if len(at[i])!=0:
			string="("
			for j in at[i]:
				string+=j
				string+="#"
			string=string[:-1]
			string+=")"
		else:
			string=""
		at_str[i]=string
		
	f = open('task1.csv', 'w')	
	for i in at_str:
		f.write(at_str[i])
		f.write(",")
		f.write(i)
		f.write("\n")
	f.close()
	
def check_ac():
	ac_food=[]	
	ac_service=[]
	ac_price=[]
	ac_ambience=[]
	ac_others=[]
	lines = [line.split(',') for line in open('ac_food_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_food.append(adj)
	lines = [line.split(',') for line in open('ac_service_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_service.append(adj)
	lines = [line.split(',') for line in open('ac_price_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_price.append(adj)
	lines = [line.split(',') for line in open('ac_ambience_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_ambience.append(adj)
	lines = [line.split(',') for line in open('ac_others_synsets.csv')]	
	for i in lines:
		adj=i[0][:-1]
		ac_others.append(adj)	
	
	lines = [line.split(',') for line in open('marked.csv')]
	at_ac=dict()
	for i in lines:	#we have marked each AT to AC handful in form of 1~5
		if i[1]=="1":
			at_ac[i[0]]="food"
		elif i[1]=="2":
			at_ac[i[0]]="service"
		elif i[1]=="3":
			at_ac[i[0]]="price"
		elif i[1]=="4":
			at_ac[i[0]]="ambience"
		else:
			at_ac[i[0]]="anecdotes/miscellaneous"

	task3=dict()
	term_sentence=dict()
	lines = [line.split(',') for line in open('task1.csv')]	
	for i in lines:
		str1=i[1:]
		str2=""
		for k in str1:
			str2+=k
			str2+=","
		str2=str2[0:-2]
		task3[str2]=[]
		if i[0]=="":
			continue
		else:
			list1=re.findall("\((\w+)\#",i[0])
			for j in list1:
				if j in at_ac:
					task3[str2].append(at_ac[j])
			list1=re.findall("\((\w+\s\w+)\#",i[0])
			for j in list1:
				if j in at_ac:
					task3[str2].append(at_ac[j])
			list1=re.findall("\((\w+\s\w+\s\w+)\#",i[0])
			for j in list1:
				if j in at_ac:
					task3[str2].append(at_ac[j])
			list1=re.findall("\((\w+\s\w+\s\w+\s\w+)\#",i[0])
			for j in list1:
				if j in at_ac:
					task3[str2].append(at_ac[j])
			list1=re.findall("\((\w+\s\w+\s\w+\s\w+\s\w+)\#",i[0])
			for j in list1:
				if j in at_ac:
					task3[str2].append(at_ac[j])
	for i in task3:
		if len(task3[i])==0:
			str3=TextBlob(i)
			tags=str3.tags
			for j in tags:
				if j[1] in ("JJ" , "JJR" , "JJS"):
					if j[0] in ac_food:
						task3[i].append("food")
					elif j[0] in ac_service:
						task3[i].append("service")
					elif j[0] in ac_price:
						task3[i].append("price")
					elif j[0] in ac_ambience:
						task3[i].append("ambience")
					else:
						task3[i].append("anecdotes/miscellaneous")
	for i in task3:	#if one sentence have no AC, then put "anecdotes/miscellaneous" as its AC
		if len(task3[i])==0:
			task3[i].append("anecdotes/miscellaneous")
	for i in task3:
		task3[i]=set(task3[i])
		task3[i]=list(task3[i])
	ac_str=dict()	#adjust the form
	for i in task3:
		if len(task3[i])!=0:
			string="("
			for j in task3[i]:
				string+=j
				string+="#"
			string=string[:-1]
			string+=")"
		else:
			string=""
		ac_str[i]=string
	f = open('task3.csv', 'w')	
	for i in ac_str:
		f.write(ac_str[i])
		f.write(",")
		f.write(i)
		f.write("\n")
	f.close()	
	
if __name__ == '__main__':
	at_list1=train_at()
	at_list2=test_at(at_list1)
	at_list=at_list1+at_list2
	
	adj_ac()
	adj_wordNet()
	
	check_at(at_list)
	check_ac()